In [2]:
!pip install beautifulsoup4 requests pandas matplotlib plotly

In [1]:
import requests
import pandas as pd

In [5]:
url = f'https://graph.instagram.com/me/media?fields=id,caption,media_type,media_url,timestamp&access_token=IGQWRPYnVsc3AyaTVwZAG5LcEJ4MGxNc280ZAlpFSUY3aUxFbEFIalAyWWNOQ1BTUUh4N0luT3RXZAG93RjFtek4yV3hiYkNvZAGdpcWtxclR5cHVldmxsd2duV0dKeUtBTWh0YXM0RmpfSk56OEUwTVUwVlFTUDY0SVpldTN2MmRkNGdpUQZDZD'

In [6]:
response = requests.get(url)

In [7]:
if response.status_code == 200:
    data = response.json()
else:
    print("Error making the API request:", response.text)
    data = None

In [8]:
data

{'data': [{'id': '18049687408639342',
   'caption': 'It gets tougher everyday',
   'media_type': 'IMAGE',
   'media_url': 'https://scontent.cdninstagram.com/v/t51.29350-15/440914603_1148341009700329_8794870000605352566_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=18de74&_nc_ohc=5CeyvfsYTl0Q7kNvgFC79b1&_nc_ht=scontent.cdninstagram.com&edm=ANo9K5cEAAAA&oh=00_AfBYzJ7ICjFMv6Y-znZpjX0B7K-0l4etyKir0bocJWbYyA&oe=663BD352',
   'timestamp': '2024-05-04T10:29:34+0000'}],
 'paging': {'cursors': {'before': 'QVFIUmwwb1FXR3NxeU5ZAUWptdDBFVlJQSU1PbVprMk9GTHMxSHdwZAk42OGNyRzhab3gwWFE1TEJvTUlmdTg5MVZArU3BWMm84MHByaFZA0cTcwR1hVa2h2M3lR',
   'after': 'QVFIUmwwb1FXR3NxeU5ZAUWptdDBFVlJQSU1PbVprMk9GTHMxSHdwZAk42OGNyRzhab3gwWFE1TEJvTUlmdTg5MVZArU3BWMm84MHByaFZA0cTcwR1hVa2h2M3lR'}}}

In [38]:
if data:
    df = pd.DataFrame(data['data'])  # Assuming 'data' is a dictionary containing 'data' key with a list of records
else:
    df = pd.DataFrame(columns=['id', 'caption','mediatype','media_url','timestamp'])


In [39]:
print(df)

                   id                                            caption  \
0   18042446401450768  Well Well Well,\nif this is how waking up on t...   
1   17973556948825885  Soooo much fun with these guys!!❤️❤️\n\nOkay n...   
2   17949428459369902  Wooooow!\nI can't believe it and honestly this...   
3   17915992601505552  So,we did the secret santa a bit too early (or...   
4   17955419035656096  Ummmm a birthday post ig....\nHad a blast!\nTh...   
5   17909518853372744                    Kehndi Hundi Si @khajuriaarchit   
6   17915548409185209      My Christmas present is better than yours!😌🎄✨   
7   17933252203771714  Awkwardness is directly proportional to size e...   
8   18123764818217370  Babe we should break up, my kurti is longer th...   
9   17877087046978553                                           CARRY ON   
10  18170228749039334                       Confidence level: Kanye West   
11  17870646019798745  Throwback to 2016!\nWas so livid back then aft...   
12  18100153

In [40]:
df.to_csv('instagram_data.csv', index=False)

In [2]:
!pip install scikit-learn keras tensorflow



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, Concatenate, Dot, Activation

In [42]:
training_data = pd.read_csv('training_data.csv')

In [43]:
instagram_data = pd.read_csv('instagram_data.csv')

In [44]:
instagram_data.dropna(subset=['caption'], inplace=True)
X_train = training_data['caption'].astype(str)
y_train = training_data['depression_label']
X_test = instagram_data['caption'].astype(str)

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [46]:
max_len = 100  # Define your maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [47]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Dense(1, activation='tanh')(lstm_layer)
attention = Activation('sigmoid')(attention)
context = Dot(axes=1)([attention, lstm_layer])
output_layer = Dense(1, activation='sigmoid')(context)

model = Model(inputs=[input_layer], outputs=output_layer)

In [48]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [49]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - accuracy: 0.7858 - loss: 0.6284
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9573 - loss: 0.1389
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9796 - loss: 0.0668
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.9868 - loss: 0.0478
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.9905 - loss: 0.0326
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.9947 - loss: 0.0229
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.9943 - loss: 0.0198
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.9960 - loss: 0.0149
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - accuracy: 0.9965 - loss: 0.0139
Epoch 10/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.9976 - loss: 0.0094


In [50]:
predicted_labels = model.predict(X_test_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step


In [51]:
instagram_data['predicted_depression_label'] = predicted_labels.reshape(-1)


In [52]:
depressing_captions = instagram_data[instagram_data['predicted_depression_label'] > 0.9]['caption']


In [53]:
for caption in depressing_captions:
    print(caption)


In [54]:
instagram_data

,id,caption,media_type,media_url,timestamp,predicted_depression_label
0,18042446401450768,"Well Well Well,\nif this is how waking up on t...",CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2023-05-12T11:39:51+0000,0.413483
1,17973556948825885,Soooo much fun with these guys!!❤️❤️\n\nOkay n...,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2022-12-10T19:16:14+0000,0.000171
2,17949428459369902,Wooooow!\nI can't believe it and honestly this...,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2022-12-10T16:51:42+0000,0.097041
3,17915992601505552,"So,we did the secret santa a bit too early (or...",CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2022-03-22T15:48:47+0000,0.699847
4,17955419035656096,Ummmm a birthday post ig....\nHad a blast!\nTh...,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2022-03-13T09:01:06+0000,0.000242
5,17909518853372744,Kehndi Hundi Si @khajuriaarchit,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,2022-02-01T17:24:22+0000,0.000689
6,17915548409185209,My Christmas present is better than yours!😌🎄✨,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,2021-12-25T18:16:23+0000,0.000544
7,17933252203771714,Awkwardness is directly proportional to size e...,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2021-11-01T17:54:12+0000,0.000943
8,18123764818217370,"Babe we should break up, my kurti is longer th...",IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,2021-08-09T19:01:40+0000,0.000797
9,17877087046978553,CARRY ON,CAROUSEL_ALBUM,https://scontent.cdninstagram.com/v/t51.29350-...,2020-11-19T13:30:16+0000,0.001683
